In [66]:
import numpy as np
from sympy import *
import matplotlib.pyplot as plt
%matplotlib inline
from celmech.nbody_simulation_utilities import set_time_step,align_simulation
from celmech.nbody_simulation_utilities import get_simarchive_integration_results
import rebound as rb
from os.path import isfile
from sympy import init_printing
init_printing()

In [67]:
from celmech.secular import LaplaceLagrangeSystem
from celmech.poincare import Poincare

In [68]:
def eccMatrix(alpha, mass):
    mu1, mu2, mu3 = mass
    alpha12, alpha23 = alpha
    P1, P2, P3 = alpha12**(3/2), 1, 1/alpha23**(3/2)

    sim = rb.Simulation()
    sim.add(m=1.)
    sim.add(m=mu1, P=P1)
    sim.add(m=mu2, P=P2)
    sim.add(m=mu3, P=P3)
    ps = sim.particles
    ps[1].r = ps[1].a*(ps[1].m/3/ps[0].m)**(1/3)
    ps[2].r = ps[2].a*(ps[2].m/3/ps[0].m)**(1/3)

    sim.move_to_com()
    sim.integrator = "whfast"
    sim.dt = sim.particles[1].P/20
    sim.collision = "direct"
    align_simulation(sim)
    
    pvars = Poincare.from_Simulation(sim)
    alpha12 = pvars.particles[1].a/pvars.particles[2].a
    alpha23 = pvars.particles[2].a/pvars.particles[3].a
    alpha13 = pvars.particles[1].a/pvars.particles[3].a
#     print(f"alpha = {alpha12, alpha23, alpha13}")

    lsys = LaplaceLagrangeSystem.from_Simulation(sim)  
    M = lsys.Neccentricity_matrix
#     print(f"eccentricity matrix =")
#     print(M)
    
    return M

In [113]:
def eccMatrix_diag(alpha, mass):
    m1, m2, m3 = mass
    m_tot = m1+m2+m3
    alpha12, alpha23 = alpha
    alpha13 = alpha12*alpha23
    
    R2 = np.array([[np.sqrt(m1/m_tot), np.sqrt(m2/m_tot), np.sqrt(m3/m_tot)],
             [-np.sqrt(m3/(m1+m3)), 0, np.sqrt(m1/(m1+m3))],
             [np.sqrt(m1*m2/(m1+m3)/m_tot), -np.sqrt((m1+m3)/m_tot), np.sqrt(m2*m3/(m1+m3)/m_tot)]])
    M = eccMatrix(alpha, mass)
    
    w2 = 2*m2/(m1+m3) * (m1/(np.pi*(1-alpha23)**2) + m3/(np.pi*(1-alpha12)**2)) + 2*(m1+m3)/(np.pi*(1-alpha13)**2)
    w3 = 2*m_tot/(m1+m3) * (m1/(np.pi*(1-alpha12)**2) + m3/(np.pi*(1-alpha23)**2))
    k = np.sqrt(m1*m2*m3*m_tot)/(m1+m3) * (2/(np.pi*(1-alpha23)**2) - 2/(np.pi*(1-alpha12)**2))
    phi = 1/2*np.arctan2(2*k, w2-w3) - np.pi/2
    R1 = np.array([[1, 0, 0],
                [0, np.cos(phi), np.sin(phi)],
                [0, -np.sin(phi), np.cos(phi)]])
    R = np.matmul(R1, R2)
    print(np.matmul(np.matmul(R2, M), np.transpose(R2)))
    print(f'phi={phi}')
    
#     vals, k = np.linalg.eigh(M)
    
    return np.matmul(np.matmul(R, M), np.transpose(R))

In [117]:
x = eccMatrix_diag((0.95,0.96), (1e-8, 1e-8, 1e-8))
print(x)
# print(f"sine = {np.sin(psi)}, cosine = {np.cos(psi)}")
# print(f"sine = {k/delta}, cosine = {np.sqrt(1-(k/delta)**2)}")

[[-7.90497353e-08  6.66435985e-08  1.10587531e-08]
 [ 6.66435985e-08 -7.17814235e-06 -1.57219897e-06]
 [ 1.10587531e-08 -1.57219897e-06 -1.45622280e-05]]
phi=-0.2351222803666817
[[-7.90497353e-08  6.22336892e-08  2.62799006e-08]
 [ 6.22336892e-08 -6.86651576e-06  2.71331965e-07]
 [ 2.62799006e-08  2.71331965e-07 -1.48738546e-05]]


In [77]:
k, delta

In [74]:
np.arctan(0.23)

In [75]:
np.arctan2(0.23, 1)